<a href="https://colab.research.google.com/github/shir994/misis_seminars/blob/master/Calorimeter_seminar_misis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# let's download today dataset
!wget -O MC https://cernbox.cern.ch/index.php/s/R2RtKIGwk0j4s16
!wget -O TT https://cernbox.cern.ch/index.php/s/wENqqgAIhTvKO0o

def convert_to_direct_link(fname):
  def get_direct_link(fname):
    with open(fname) as f:
      lines = f.readlines()
      for l in lines:
        if 'downloadURL' in l:
          parts = l.split()
          for p in parts:
            if p[:5] == 'value':
              return p.split('"')[-2]
      raise KeyError("downloadURL not found")
  
  link = get_direct_link(fname)
  with open(fname, 'w') as f:
    f.write(link)

convert_to_direct_link('MC')
convert_to_direct_link('TT')
!wget -O MC_misis.csv `cat MC`
!wget -O TT_misis.hdf `cat TT`

In [0]:
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib import pylab as plt 
#import root_numpy
import pandas as pd
import numpy
#import seaborn
%matplotlib inline
#%matplotlib notebook
import numpy as np
import os

In [0]:
import pickle

In [0]:
plt.rcParams['axes.facecolor'] = 'white'
plt.grid(c='black')

In [0]:
num_showers = 30000

In [0]:
MC_df = pd.read_csv("MC_misis.csv")

In [0]:
TT_df = pd.read_hdf('TT_misis.hdf', 'df')
TT_df.shape

# Plot shower view

In [0]:
Z_LEFT = -3256.6700
END_OF_BRICK = Z_LEFT + 0.56 * 30
#Z_RIGHT = -3245.6700
X_HALF_SIZE = 6.45
Y_HALF_SIZE = 5.25

def create_plotting_df(df, dZ):
    # mind the order!
    d0 = pd.DataFrame([
                df['Z'],
                df['X'],
                df['Y']],
                index=['z', 'x', 'y']).T
    numtracks = d0.shape[0]
    dd = pd.DataFrame([np.ones(len(df['SX'])), df['SX'], df['SY']], index=['z', 'x', 'y']).T
    dd *= dZ
    d1 = d0 + dd
    return d0, d1

def plot_mc(emulsion, id=0):
    dZ = 0.0315
    df = emulsion.iloc[id]
    d0, d1 = create_plotting_df(df, dZ)
    C = plt.cm.Reds(0.5)
    lc_1 = Line3DCollection(list(zip(d0.values, d1.values)), colors=C, alpha=0.9, lw=2)
    
    fig = plt.figure(figsize=(12,8))
    ax = fig.gca(projection='3d')
    ax.view_init(azim=-90, elev=0)
    ax.add_collection3d(lc_1)
    
    # mind the order!
    ax.set_xlabel("z")
    ax.set_ylabel("x")
    ax.set_zlabel("y")  
    ax.set_xlim(d0['z'].min(), d0['z'].max())
    ax.set_ylim(d0['x'].min(), d0['x'].max())
    ax.set_zlim(d0['y'].min(), d0['y'].max())
    
    ax.set_xlim(Z_LEFT, END_OF_BRICK)
    ax.set_ylim(-X_HALF_SIZE, X_HALF_SIZE)
    ax.set_zlim(-Y_HALF_SIZE, Y_HALF_SIZE)
    ax.view_init(elev=30., azim=-45)
    #ax.set_xlim(-2870, -2868)
    #ax.set_zlim(-23, -22)

In [0]:
index = 10
print("E:",MC_df.iloc[index]['E'], "GeV")
plot_mc(TT_df, index)

# Shower profiles

In this task, we want to look at the shower profiles:
- How does shower profile change with the Z coordinate?
- Plot histogram of shower track X and Y coodrinate 
    as a function of X0 (for all shower, to get betters hists) 
- What do you expect to see?

In [0]:
step = 0.1315
N_FILMS = 169
# emulsion_end_coords - coordinates of emulsion films
# You need them to map each track Z coordinate to the index of emulsion film 
emulsion_end_coords = [-3256.5885 + i * step for i in range(N_FILMS)]

In [0]:
from bisect import bisect_left

In [0]:
%%time

## plate index 169 == CES tracks, not emulsion

# DF where we will accumulate all XYZ for all showers
tt_df_ravel = None
for coord in ["X", "Y", 'Z']:
    ravel = []
    TT_df[coord].apply(<YOUR CODE>)
    if tt_df_ravel is None:
        tt_df_ravel = <YOUR CODE>
    else:
        tt_df_ravel[coord] = <YOUR CODE>

tt_df_ravel['plate_index'] = <YOUR CODE>     


indeces = []
for i in range(len(TT_df)):
    indeces.extend(i * np.ones(len(TT_df.iloc[i].X)))
indeces =  np.array(indeces).astype(int)
tt_df_ravel['event_id'] = indeces

In [0]:
tt_df_ravel.head()

In [0]:
rad_length_step = 30

# Radiation lengh in number of films
rad_length_in_n_plates = (N_FILMS - 1) / rad_length_step

In [0]:
def plot_distribution(coord, start_point, end_point):
    """coord :str: - axis of selection, ie X or Y,
        start_point - first X0 to plot,
        end_point - last X0 to plot
    """
    selection_indces = [int(rad_length_in_n_plates * i) for i in range(start_point, end_point)]
    labels = [str(i) + "X0" for i in range(start_point, end_point)]
    # Plot the histogram of shower profile here, using variables above
    for label, p_index in zip(labels, selection_indeces):
      cut_df = tt_df_ravel[<YOUR CODE>]
      plt.hist(<YOUR CODE>)      

    plt.legend()
    plt.xlabel(coord + ", cm", fontsize=23)

In [0]:
plt.figure(figsize=(21,12))
for index, (start, end) in enumerate(zip([1,11,21], [11,21,31])):
    plt.subplot(2,3,index+1)
    plot_distribution('X', start, end)
    plt.subplot(2,3,index+4)
    plot_distribution('Y', start, end)

# Plot shower N-vs-X0

In this task, we want to see the dependance of number of tracks in shower as a funciton of X0:
- Plot histogram of number of tracks vs X0 (every 1 X0). Plot 1 standard deviation in each point. 
- What do you expect to see?
- How do you thing the shape will change for different energies?

In [0]:
# Get total number of tracks in shower
total_number_of_tracks_in_shower = <YOUR CODE>

mean_n_hits_and_std = []
# Save mean number of tracks and std in the list above
for p_index in range(1, N_FILMS):
    cut_df = tt_df_ravel[<YOUR CODE>]
    n_hits = <YOUR CODE>
    mean_n_hits_and_std.append([n_hits.mean(), n_hits.std()])

mean_n_hits_and_std = np.array(mean_n_hits_and_std)    

In [0]:
import matplotlib.patches as mpatches

In [0]:
x_ticks = [int(rad_length_in_n_plates * i) for i in range(1, rad_length_step + 1)]
labels = [str(i) for i in range(1, rad_length_step + 1)]

In [0]:
plt.figure(figsize=(16,8))
# Plot the distribution
<YOUR CODE>

plt.grid(color='grey')
plt.ylabel("Number of tracks, fraction", fontsize=23)
plt.xlabel("Distance from the origin, X0", fontsize=23)
plt.xticks(x_ticks, labels);
red_patch = mpatches.Patch(color='green', label='$1 \sigma$ deviation', linewidth=10)
plt.legend(handles=[red_patch], fontsize = 'xx-large')



What do we expect from the simple theory?

# Calibration

We know that for EM shower $E \sim N$, where N - is the number of particles in a shower.
Let see if this relationship holds in our case.

In [0]:
# Plot scatter plot of N in shower vs E of initial electron


In [0]:
plt.figure(figsize=(16,8))
plt.scatter(MC_df, total_number_of_tracks_in_shower)
plt.grid(color='grey')
plt.ylabel("Number of tracks", fontsize=23)
plt.xlabel("Energy, GeV", fontsize=23)

So, we want to calibrate our "detector" to predict energy,  based on its observations.
For this, we can simply split our data to train and test - on train we will fit our model, on test we will validate our result.

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(total_number_of_tracks_in_shower, MC_df.values, test_size=0.1,
                                                   random_state=1543)
from sklearn.linear_model import LinearRegression, RANSACRegressor

Using imported regressors, fit them on traning data and then plot the above plot with overlayed regression on it.
Sklearn already provides you with evrything you need. Basically, LinearRegression is fitting exactly
$$
E = a * N + b, \\
$$
where N - number of hits in shower.

During fitting, it minimises $MSE: \mathbb{E}[(E_{true} - E_{reco})^2$]

But, as you can see, we have some outliers. Linear regression is proned to overfitting and thus outliers affect fitting a lot. Sometimes, its good to use methods, that are stable to outliers. here, I give an example of RANSAC regressor.

In [0]:
<YOUR CODE>

In [0]:
plt.figure(figsize=(16,8))
plt.scatter(y_test, X_test, label='test data')

# he we simply need to evalutae fitted relation E = a * N +b at N = 0 and 2000
plt.plot([<YOUR CODE>, <YOUR CODE> * 2000], [0, 2000], linewidth=5, c='red', label='fit')
plt.plot([<YOUR CODE>,
          <YOUR CODE> * 2000], [0, 2000], linewidth=3, c='green', label='fit')

plt.grid()
plt.ylabel("Number of tracks", fontsize=23)
plt.xlabel("Energy, GeV", fontsize=23)
plt.xlim(4.9,6.1)
plt.legend()

How can we measure performance of our fit?

In [0]:
y_pred = lr.predict(X_test.reshape(-1,1))[:,0]

In [0]:
from scipy.stats import binned_statistic

Of course, the one of the ways to measure the perforamnce is check Redisidual sum of squares (MSE error). 
Ideal estimator will be unbiased and have zero variance.

In [0]:
bins_width = list(np.arange(5, 6, 0.1))
val_bins, bins, _ = binned_statistic(y_test[:,0], y_pred, "mean", bins=bins_width)
err_bins, bins, _ = binned_statistic(y_test[:,0], y_pred, "std", bins=bins_width)

In [0]:
plt.figure(figsize = (16, 10))
displayed_lr = LinearRegression()
displayed_lr.fit(y_test, y_pred)


plt.errorbar((bins[:-1] + bins[1:]) / 2, val_bins, err_bins, np.diff(bins_width) / 2, fmt='o')
plt.plot([0, 6], [displayed_lr.intercept_, displayed_lr.intercept_ + displayed_lr.coef_[0] * 6])
plt.xlim(4.9, 6.1)
plt.ylim(4.9, 6.1)
plt.xlabel("True energy, GeV",fontsize=22)
plt.ylabel("Reco energy, GeV", fontsize=22)
plt.tick_params(labelsize=15)
plt.grid(color='grey')

In [0]:
from sklearn.metrics import mean_squared_error
print("MSE:{}".format(mean_squared_error(y_test, y_pred)))

OK, MSE seems ok. Lets look at the distributions of the answers

In [0]:
plt.figure(figsize = (16,10))
# Plot the histogram of the residuals. What do you observe?
plt.hist(<YOUR CODE>);

plt.xlabel("E_true - E_reco, GeV", fontsize=23);
plt.ylabel("a.u.", fontsize=23);
plt.grid(color='grey')

Why our distribution is skewed?

Ok, now, once we have plotted bias, we can check how well the alorithms performs in energy resolution - the main characterictics of any calorimiter.

From the lecture, we know, that stochastic term of energy resolution is proportional to:
$$
\sigma_{E} \sim \frac{1}{\sqrt{E}}
$$

Lets validate this in practice.

In [0]:
from scipy.optimize import curve_fit


# here we need to return exactly the curve we want to fit: in our case a + b * \sqrt{1}(E)
def s_root(x, a, b):
    return <YOUR CODE>

def get_resolution(y_pred, y_true, cut=20):
    # Binning the data
    bins_width = list(np.arange(5, 6, 0.1))
    
    data = <WHAT we want to plot on y axis> - energy resolution
    x_bins = <WHAT we want to plot on x axis> - true energy

    
    resol_bins, bins, _ = binned_statistic(x_bins, data, "std", bins=bins_width)
    
    # Ok, but how do we estimate error on stadard deviation?
    # we will use bootstrap!
    # Ok, now using lambda function, extract error on standard deviation
    
    resol_error, bins, _ = binned_statistic(x_bins, data, <YOUR CODE>)

    pw, cov = curve_fit(s_root, ((bins[:-1] + bins[1:]) / 2), resol_bins)
    return bins, bins_width, resol_bins, resol_error, pw

In [0]:
# With cut
bins, bins_width, resol_bins, resol_error, pw = get_resolution(lr.predict(X_test.reshape(-1,1)),
              y_test, cut=0.5)

In [0]:
plt.figure(figsize = (16,10))

plt.errorbar((bins[:-1] + bins[1:]) / 2, resol_bins, resol_error, np.diff(bins_width) / 2, fmt='o',
             label='simulation data')
plt.plot(bins, s_root(bins_width, *pw), label='1 / $\sqrt{E}$ fit')


plt.ylabel("Energy resolution",fontsize=22)
plt.xlabel("Energy, GeV", fontsize=22)
plt.tick_params(labelsize=15)
plt.legend()
plt.grid(color='grey')

# Energy resolution and shower leakage

Ok, remember the graph for the number of tracks versus X0? In reality, you can not build calorimeter as big as you want, since you always have geometrical constraints. Thus, in the final part, lets plot the energy resolution as a function of the size of calorimiter in X0.

In [0]:
# Fill the n_hits matrix with will have shape (num_showers, N_FILMS)
# it will contain number of track for each shower in layer X

n_hits = np.zeros((num_showers, N_FILMS))
counter = tt_df_ravel.groupby(<YOUR CODE>).<YOUR CODE>
for index in range(num_showers):
    plate_index = <YOUR CODE>
    counts = <YOUR CODE>
    n_hits[index, plate_index - 1] = counts

In [0]:
x_ticks = [int(rad_length_in_n_plates * i) for i in range(1, rad_length_step + 1)]
labels = [str(i) for i in range(1, rad_length_step + 1)]

# Now we want to calculate cumulative sum - number of tracks in each shower UP TO layer X
stats_by_rad_length = <YOUR CODE>

`resolution_results` is matrix of shape (num_showers, len_of_X0_range)

In [0]:
# here we repeat the fitting and prediction for each of the X0 cut
resolution_results = []
cut = None
for index in range(30):
    X_train, X_test, y_train, y_test = train_test_split(stats_by_rad_length[:, index], MC_df.values,
                                                        test_size=0.1, random_state=1543)
    lr = LinearRegression()
    lr.fit(X_train.reshape(-1,1), y_train)
    y_pred = lr.predict(X_test.reshape(-1,1))[:,0]
    
    data = ((y_pred.reshape(-1,1) - y_test) / y_test)[:,0]
    if cut is not None:
        data = data[np.abs(data) < cut]
    resolution_results.append(np.std(data))

In [0]:
plt.figure(figsize = (16,10))
plt.scatter(range(1, 31), resolution_results)

plt.ylabel("Energy resolution",fontsize=22)
plt.xlabel("Distance from the origin, X0", fontsize=23)
plt.tick_params(labelsize=15)
plt.grid(color='grey')

In [0]:
plt.figure(figsize=(18,12))
for index, x_0 in enumerate([0, 5, 10, 15, 20, 29]):
    plt.subplot(2,3,index+1)
    X_train, X_test, y_train, y_test = train_test_split(stats_by_rad_length[:, x_0], MC_df.values,
                                                    test_size=0.1, random_state=1543)
    lr.fit(X_train.reshape(-1,1), y_train)
    y_pred = lr.predict(X_test.reshape(-1,1))[:,0]
    plt.hist(y_test, alpha=0.5, label='y_true', bins=30)
    plt.hist(y_pred, alpha=0.8, label='y_pred', bins=30)
    plt.legend()
    plt.title("Length: {} X0".format(x_0))
    plt.xlabel("Energy, GeV")